### **Disjunctive programming**

In [6]:
import matplotlib.pyplot as plt
import pandas as pd

from pyomo.environ import *
from pyomo.gdp import *
import pandas as pd

In [7]:
# load data as dictionary of components
# component data consists of cost and composition 
comp_data = {
    "A": {"cost": 2.0, "Vit A": 0.5, "Vit B": 0.2},
    "B": {"cost": 2.0, "Vit A": 0.4, "Vit B": 0.1},
    "C": {"cost": 5.0, "Vit A": 0.3, "Vit B": 0.3},
}

pd.DataFrame.from_dict(comp_data, orient='index')

,cost,Vit A,Vit B
A,2.0,0.5,0.2
B,2.0,0.4,0.1
C,5.0,0.3,0.3


##### Product Composition Requirements

Find the lowest cost blend

* Vit A: less than 0.4
* Vit B: greater than 0.2

Your code should be able to accept alternative specification for data and product requirements.


In [8]:
prod_req = {
    "Vit A": {"lb": 0.0, "ub": 0.4},
    "Vit B": {"lb": 0.2, "ub": 1.0},
}
pd.DataFrame.from_dict(prod_req, orient='index')

,lb,ub
Vit A,0.0,0.4
Vit B,0.2,1.0


##### Component Compatibility

For this application, we consider an additional type of constraint specifying the incompatability of certain blends of components. For example, suppose we have a constraint:

* A and B cannot be mixed together in the final product

The constraint is specified by creating a list of incompatabile pairs.

In [9]:
excl_pairs = [("A", "B")] # Exclusive pair

In [14]:
# Solving the problem neglecting exclusive pair

m = ConcreteModel()

# define sets that will be used to index decision variables and constraints
# remember to use initialize keyword
m.comp = Set(initialize=comp_data.keys())
m.req = Set(initialize=prod_req.keys())

# decision variables
m.x = Var(m.comp, domain=NonNegativeReals)

# objective function
m.cost = Objective(expr=sum(m.x[c]*comp_data[c]["cost"] for c in m.comp), sense=minimize)

# structural constraints
m.massfraction = Constraint(expr=sum(m.x[c] for c in m.comp)==1)

# composition constraints
m.lb = Constraint(m.req, rule=lambda m, r: sum(m.x[c]*comp_data[c][r] for c in m.comp) >= prod_req[r]["lb"])
m.ub = Constraint(m.req, rule=lambda m, r: sum(m.x[c]*comp_data[c][r] for c in m.comp) <= prod_req[r]["ub"])

solver = SolverFactory('CBC')
solver.solve(m)

for c in m.comp:
    print(f"{c} = {m.x[c]()}")

A = 0.3333333333333336
B = 0.33333333333333315
C = 0.33333333333333326
